In [1]:
import os
import pandas as pd
import FlowCal
import flowsom as fs
from sklearn_extra.cluster import CLARA
import phenograph
import numpy as np
import re
from flowio import create_fcs
from scipy.stats import gmean
from fcsy import DataFrame
from fcsy import write_fcs


In [7]:
# Definir los directorios de entrada y salida
work_dir = r"C:/Users/Usuario/OneDrive/Escritorio/Palex/Muestras TFM/SAMPLES_SCAFOLD"
input_dir = work_dir + '/Muestras_cluster'
output_dir = work_dir + '/clusters_clara'
flowsom_dir = work_dir + '/clusters_flowsom'
phenograph_dir = work_dir + '/clusters_phenograph'
clara_dir = work_dir + '/clusters_clara'
clara_controles_sep_dir = work_dir + '/clusters_clara_controles_sep'
clara_unidos_dir = work_dir + '/clusters_clara_unidos'

In [3]:
# Función para limpiar nombres de variables
def limpiar_nombre_variable(nombre):
    # Reemplazar caracteres no deseados con un guion bajo o eliminarlos (en este caso se eliminan)
    nombre_limpio = re.sub(r'[^\w]', '_', nombre)
    return nombre_limpio

# Clustering

In [4]:
# Cogemos los labels antes de normalizar para añadirlos al final
norm_dir = work_dir + '/Controles_muestras_normalizados_no_concat'
archivos_csv = [f for f in os.listdir(norm_dir) if f.endswith('.csv') and f.startswith('PRETR')]
archivos_controles = [f for f in os.listdir(norm_dir) if f.endswith('.csv') and f.startswith('C')]

## CLARA

In [8]:
num_clusters = 200

# Crear el directorio de salida si no existe
if not os.path.exists(clara_dir):
    os.makedirs(clara_dir)

for archivo in archivos_csv:
    # Construir la ruta completa al archivo
    archivo_path = os.path.join(norm_dir, archivo)

    # Leer el archivo CSV
    df = pd.read_csv(archivo_path)

    # Aplicar CLARA al DataFrame
    clara = CLARA(n_clusters=num_clusters, random_state=0)
    clara.fit(df)

    # Añadir los clusters al DataFrame
    df['cluster'] = clara.labels_

    # Guardar los resultados en un archivo CSV
    archivo_nombre = os.path.splitext(os.path.basename(archivo))[0]
    nombre_salida = os.path.join(clara_dir, f"{archivo_nombre}_clusters.csv")
    df.to_csv(nombre_salida, index=False)


Clusterizacion de controles conjuntamente

In [8]:
num_clusters = 300

# Crear el directorio de salida si no existe
if not os.path.exists(clara_controles_sep_dir):
    os.makedirs(clara_controles_sep_dir)

df_controles_total = []
for archivo in archivos_controles:
    archivo_path = os.path.join(norm_dir, archivo)
    df_temp = pd.read_csv(archivo_path)
    # Añadimos una columna para identificar su procedencia
    df_temp['Archivo'] = archivo
    df_controles_total.append(df_temp)

df_controles_total = pd.concat(df_controles_total, ignore_index=True)

# Aplicar CLARA de forma conjunta a todos los controles
clara_controls = CLARA(n_clusters=num_clusters, random_state=0)
clara_controls.fit(df_controles_total.drop(columns=['Archivo']))
df_controles_total['cluster'] = clara_controls.labels_

# Guardar los resultados para cada archivo de control por separado
for archivo in archivos_controles:
    df_parcial = df_controles_total[df_controles_total['Archivo'] == archivo].copy()
    df_parcial.drop(columns=['Archivo'], inplace=True)
    base_name = os.path.splitext(os.path.basename(archivo))[0]
    nombre_salida = os.path.join(clara_controles_sep_dir, f"{base_name}_clusters.csv")
    df_parcial.to_csv(nombre_salida, index=False)


archivos_no_controles = [f for f in archivos_csv if f not in archivos_controles]

for archivo in archivos_no_controles:
    archivo_path = os.path.join(norm_dir, archivo)
    df = pd.read_csv(archivo_path)

    clara_muestras = CLARA(n_clusters=num_clusters, random_state=0)
    clara_muestras.fit(df)

    df['cluster'] = clara_muestras.labels_
    archivo_nombre = os.path.splitext(os.path.basename(archivo))[0]
    nombre_salida = os.path.join(clara_controles_sep_dir, f"{archivo_nombre}_clusters.csv")
    df.to_csv(nombre_salida, index=False)

MemoryError: Unable to allocate 2.68 GiB for an array with shape (1200000, 300) and data type float64

## CLARA junto

In [5]:
num_clusters = 200

# Crear el directorio de salida si no existe
if not os.path.exists(clara_unidos_dir):
    os.makedirs(clara_unidos_dir)

# Leer y unir todos los archivos
dfs = []
for archivo in archivos_csv:
    archivo_path = os.path.join(norm_dir, archivo)
    df_temp = pd.read_csv(archivo_path)
    df_temp['Source_File'] = archivo  # Para saber de qué archivo viene
    dfs.append(df_temp)

df_all = pd.concat(dfs, ignore_index=True)

# Aplicar CLARA a todo el conjunto
clara = CLARA(n_clusters=num_clusters, random_state=0)
clara.fit(df_all.drop(columns=['Source_File']))

df_all['cluster'] = clara.labels_

# Separar resultados y guardarlos en archivos
for archivo in archivos_csv:
    df_filtro = df_all[df_all['Source_File'] == archivo].drop(columns=['Source_File'])
    archivo_nombre = os.path.splitext(os.path.basename(archivo))[0]
    nombre_salida = os.path.join(clara_unidos_dir, f"{archivo_nombre}_clusters.csv")
    df_filtro.to_csv(nombre_salida, index=False)

## Phenograph

In [6]:
# Crear el directorio de salida si no existe
if not os.path.exists(phenograph_dir):
    os.makedirs(phenograph_dir)

for archivo in archivos_csv:
    # Construir la ruta completa al archivo
    archivo_path = os.path.join(norm_dir, archivo)

    # Leer el archivo CSV
    df = pd.read_csv(archivo_path)

    # Aplicar Phenograph al DataFrame
    clusters, _, _ = phenograph.cluster(df.values, n_jobs=-1, k = 30)

    # Añadir los clusters al DataFrame
    df['cluster'] = clusters

    # Guardar los resultados en un archivo CSV
    archivo_nombre = os.path.splitext(os.path.basename(archivo))[0]
    nombre_salida = os.path.join(phenograph_dir, f"{archivo_nombre}_clusters.csv")
    df.to_csv(nombre_salida, index=False)

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 13.559554100036621 seconds
Jaccard graph constructed in 10.139896631240845 seconds
Wrote graph to binary file in 1.7510058879852295 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.948305
Louvain completed 21 runs in 133.14803791046143 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 160.35065245628357 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 14.623748064041138 seconds
Jaccard graph constructed in 8.809340476989746 seconds
Wrote graph to binary file in 2.2291829586029053 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.937557
Louvain completed 21 runs in 115.22807788848877 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 142.74029111862183 seconds
Finding 30 nearest neighbors using minkowski metr